# Lab Assignment 2 
## Signature
-----
Alexandre Dufresne-Nappert

20948586

## Tasks
-----

 1. Antiferromagnetic case (J = –1) — Reproduce Figure 10.2 for an antiferromagnet and compare with the
 ferromagnetic case.

 2. Equilibrium analysis — Test if 2000 sweeps are sufficient to reach equilibrium by plotting energy and
 magnetization vs. sweep number at several temperatures.
 
 3. External magnetic field — Modify the model to include a field term (with $g\cdot \mu_B\cdot H = 1$) and study its impact
 on energy, magnetization, heat capacity, and susceptibility.

## Deliverables
-----
Submit a Jupyter Notebook that includes:
- Code (based on Listing 10.1)
- Plots and numerical analysis for each task
- Short discussion of results and physical interpretation


# Code Definition
----
## Notes
-----
I've reorganized the code so that functions are defined at the beginning, I've also optimized the code by having variables passed through by reference instead of using global accesses. 

Most importantly I added the Numba package to make the code compile to C using JIT compilation (Just In Time compilation) so that the code can run a lot faster

In [7]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import rand, randint, permutation
from numba import njit

In [ ]:
# Ising.py Function Definitions

# Function to change in dices to subscripts ---------------------------------
@njit
def ind2sub(indx, L):
    column = np.floor(indx / L)
    row = indx - column * L
    return row, column

# Metropolis algorithm function --------------------------------------------
@njit
def Metropolis(rowsub, colsub, trial, Spins, eqbmstart, N, L, T, J):
    inverseT = 1 / T
    Energy = 0
    for counter in range(N):  # for every spin site

        # identify row and its neighbours
        r = int(rowsub[counter])

        lo = r - 1
        if lo < 0:
            lo = L - 1

        hi = r + 1
        if hi > L - 1:
            hi = 0

        c = int(colsub[counter])  # identify column and its neighbours

        lf = c - 1
        if lf < 0:
            lf = L - 1

        rt = c + 1
        if rt > L - 1:
            rt = 0

        # Sum neighbouring spins
        SS = Spins[lo, c] + Spins[hi, c] + Spins[r, lf] + Spins[r, rt]
        # Calculate energy change if spin were to be flipped

        DeltaE = 2 * J * SS * Spins[r, c]
        # Metropolis algorithm
        if DeltaE < 0 or rand() < np.exp(-DeltaE * inverseT):  # flip spins
            Spins[r, c] = -Spins[r, c]
            if trial >= eqbmstart:  # calculate equilibrium energy
                Energy = Energy - 0.5 * J * SS * Spins[r, c]
    # end of for counter in range(N)
    return Spins, Energy

@njit
def RunMonteCarloTrials(trials, eqbmstart, neq, N, L, T, J, Spins, M, E):
    # Perform Monte-Carlo trials -----------------------------------------------
    for trial in range(trials):
        indx = permutation(N)  # random permutation of 1 to N
        rowsub, colsub = ind2sub(indx, L)  # convert indx to subscripts

        # Flip spins using the Metropolis algorithm
        Spins, Energy = Metropolis(rowsub, colsub, trial, Spins, eqbmstart, N, L, T, J)

        # record energy and magnetisation
        if trial >= eqbmstart:
            t = trial - eqbmstart
            E[t] = Energy  # energy
            M[t] = np.sum(Spins)  # magnetisation

    # end of for trial in range(trials) ----------------------------------------
    Eav = np.mean(E) / N  # average equilibrium energy per spin
    Cv = (np.var(E) / T**2) / N  # specific heat capacity per spin
    Mav = np.mean(M) / N  # magnetisation per spin
    Chi = (np.var(M) / T) / N  # magnetic susceptibility per spin

    return Eav, Cv, Mav, Chi

In [ ]:
# Ising.py

# Basic data ---------------------------------------------------------------
L = 100  # Number of spin sites on a side.
N = L**2  # Total number of spin sites.
J = 1  # Exchange constant +(-) ve,ferro(antiferro)magnet
T = 2.3  # Temperature
eqbmstart = 2000  # Number of trials to establish equilibrium
neq = 1000  # Number of equilibrium trials
trials = eqbmstart + neq  # Total number of trials

# Initialisation -----------------------------------------------------------
Spins = -1 + 2 * randint(2, size=(L, L))  # lxl matrix of spins (+/-1)
M = np.zeros(neq)  # storage space for magnetisation
E = np.zeros(neq)  # storage space for energy

print(RunMonteCarloTrials(trials, eqbmstart, neq, N, L, T, J, Spins, M, E))

(-1.13e-05, 0.046602915519848784, 0.21460479999999998, 55.21759920417397)
